In [1]:
# rank the obtained results using the *.log files
import os
import pandas as pd
import numpy as np

In [2]:
source = "10"
targetdir = '../../data/' + source + "/"
filelist = sorted(os.listdir(targetdir))

In [3]:
filelist

['1.data', '2.data', '3.data', '4.data', '5.data', '6.data', '7.data']

In [4]:
# Create dataframe from files
df = pd.DataFrame()

for file in filelist:
    filename = targetdir+file
    col_name = [file]
    temp_df = pd.read_csv(filename,names=col_name)    
    df = pd.concat([df, temp_df], axis=1)
    
# Look at the data
df.head()

,1.data,2.data,3.data,4.data,5.data,6.data,7.data
0,0.7981,0.9064,1.0000,1.0000,0.9103,1.0000,0.6492
1,0.7981,0.9064,1.0001,-1.0000,0.9103,-1.0000,0.6492
2,0.7981,0.9064,1.0002,-1.0000,0.9103,-1.0000,0.6492
3,0.6604,0.7501,0.8219,0.8276,0.7533,0.5203,0.5373
4,0.6604,0.7501,0.8219,0.8276,0.7533,0.5203,0.5373


In [5]:
# Clip values > 1 with 1 and ignore 0s
df.mask(df > 1, 1, inplace=True)
df.mask(df <= 0, np.NaN, inplace=True)

In [6]:
# Count NaN values
df.isna().sum()

1.data      0
2.data      0
3.data      0
4.data    324
5.data      0
6.data    144
7.data      0
dtype: int64

In [7]:
df

,1.data,2.data,3.data,4.data,5.data,6.data,7.data
0,0.7981,0.9064,1.0000,1.0000,0.9103,1.0000,0.6492
1,0.7981,0.9064,1.0000,NaN,0.9103,NaN,0.6492
2,0.7981,0.9064,1.0000,NaN,0.9103,NaN,0.6492
3,0.6604,0.7501,0.8219,0.8276,0.7533,0.5203,0.5373
4,0.6604,0.7501,0.8219,0.8276,0.7533,0.5203,0.5373
...,...,...,...,...,...,...,...
724,0.7393,0.7869,0.9528,NaN,0.7734,0.5320,0.5525
725,0.7393,0.7869,0.9528,NaN,0.7734,0.5320,0.5525
726,0.6306,0.6712,0.7977,1.0000,0.6597,0.4538,0.4713
727,0.6306,0.6712,0.7977,1.0000,0.6597,0.4538,0.4713


In [8]:
# Save processed dataframe as csv file
df.to_csv("../../data/processed/asymmetric/" + source +".csv",index=False)

In [9]:
# Creating ranked dataframe
ranked_df = pd.DataFrame()
stats_df = pd.DataFrame()

In [10]:
# going through every column
for column in df:  
    wwtp = column[0]
    
    # In every column, drop na values
    asym_column = df[column].dropna()
    
    # and calculate individual tao
    tao = len(asym_column)
    
    # calculate mean 
    avg_eff = round(asym_column.mean(),3)
    
    # calculate max
    max_eff = round(asym_column.max(),3)
    
    # calculate min
    min_eff = round(asym_column.min(),3)
    
    # calculate amplitude 
    amplitude = round((max_eff - min_eff)*100,2)
    
    amp_str = "Amplitude (max-min)(%)"
    
    # print stats results
    print("WWTP",wwtp,
          "\nMean =",avg_eff,"Maximum =",max_eff,"Minimum =",min_eff, amp_str,"=",amplitude)
    
    stats_df = stats_df.append({ 'WWTP': wwtp,
        "Mean": avg_eff, "Maximum" : max_eff, "Minimum": min_eff, amp_str: amplitude},ignore_index=True)
    
    # Calculating Sk sum of factors
    Sk = round(asym_column.sum(),3)    
    
    # Calculating ek sum of factors of 1 (or above if errors in calculation)
    ek = asym_column >= 1
    ek = ek.sum()
        
    # Calculating R1k ek/tao
    R1k = round(ek/tao,3)
    
    # Calculate R2k
    if tao != ek:
        R2k = (Sk - ek)/(tao - ek)
    elif R1k == 1:
        R2k = 0
        
    R2k = round(R2k,3)
    
    # Printing results    
    print("tao =",tao,"| ek =",ek,"| R1k =",R1k, "| Sk =",Sk, "| R2k =",R2k,"\n")    
    
    # Populate ranking dataframe using pd.df.append
    # Using unicode to name columns with super and subscripts
    R1k_col = 'R\u00B9\u2096\u2080'
    R2k_col = 'R\u00B2\u2096\u2080'
    ranked_df = ranked_df.append({ R2k_col:R2k, R1k_col: R1k,'WWTP': wwtp},ignore_index=True)  

WWTP 1 
Mean = 0.714 Maximum = 0.912 Minimum = 0.552 Amplitude (max-min)(%) = 36.0
tao = 729 | ek = 0 | R1k = 0.0 | Sk = 520.569 | R2k = 0.714 

WWTP 2 
Mean = 0.784 Maximum = 0.966 Minimum = 0.63 Amplitude (max-min)(%) = 33.6
tao = 729 | ek = 0 | R1k = 0.0 | Sk = 571.685 | R2k = 0.784 

WWTP 3 
Mean = 0.876 Maximum = 1.0 Minimum = 0.65 Amplitude (max-min)(%) = 35.0
tao = 729 | ek = 216 | R1k = 0.296 | Sk = 638.57 | R2k = 0.824 

WWTP 4 
Mean = 0.875 Maximum = 1.0 Minimum = 0.684 Amplitude (max-min)(%) = 31.6
tao = 405 | ek = 108 | R1k = 0.267 | Sk = 354.209 | R2k = 0.829 

WWTP 5 
Mean = 0.779 Maximum = 0.966 Minimum = 0.622 Amplitude (max-min)(%) = 34.4
tao = 729 | ek = 0 | R1k = 0.0 | Sk = 567.975 | R2k = 0.779 

WWTP 6 
Mean = 0.536 Maximum = 1.0 Minimum = 0.433 Amplitude (max-min)(%) = 56.7
tao = 585 | ek = 18 | R1k = 0.031 | Sk = 313.34 | R2k = 0.521 

WWTP 7 
Mean = 0.556 Maximum = 0.686 Minimum = 0.446 Amplitude (max-min)(%) = 24.0
tao = 729 | ek = 0 | R1k = 0.0 | Sk = 405.396 

## Ranking of WWTP

In [11]:
# Reorder columns to be usable as a results table
ranked_df = ranked_df.reindex(columns=['WWTP',R1k_col, R2k_col])

In [12]:
ranked_df

,WWTP,R¹ₖ₀,R²ₖ₀
0,1,0.000,0.714
1,2,0.000,0.784
2,3,0.296,0.824
3,4,0.267,0.829
4,5,0.000,0.779
5,6,0.031,0.521
6,7,0.000,0.556


In [13]:
import os

path = "../../results/" + source + "/asymmetric"

# Save rankings dataframe as csv file

try:
    ranked_df.to_csv(path + "/ranking.csv",index=False)
    print("Save succesful")
except:
    print("Creating folder and saving")
    os.mkdir(path)
    ranked_df.to_csv(path + "/ranking.csv",index=False)

Save succesful


## Calculate Descriptive Statistics

In [14]:
# Calculate the mean of every column
mean_mean = round(stats_df.Mean.mean(),3)
mean_max = round(stats_df.Maximum.mean(),3)
mean_min = round(stats_df.Minimum.mean(),3)
mean_amp = round(stats_df[amp_str].mean(),3)

In [15]:
# Add means to stats dataframe
stats_df = stats_df.append({ 'WWTP': "Mean", "Mean" : mean_mean, "Maximum" : mean_max, 
                            "Minimum" : mean_min, amp_str : mean_amp},ignore_index=True)

In [16]:
# Calculate the standard deviation of every column
sd_mean = round(stats_df.Mean.std(),3)
sd_max = round(stats_df.Maximum.std(),3)
sd_min = round(stats_df.Minimum.std(),3)
sd_amp = round(stats_df[amp_str].std(),3)

In [17]:
# Add means to stats dataframe
stats_df = stats_df.append({ 'WWTP': "SD", "Mean" : sd_mean, "Maximum" : sd_max, 
                            "Minimum" : sd_min, amp_str : sd_amp},ignore_index=True)

In [18]:
# Reorder columns
stats_df = stats_df.reindex(columns=["WWTP", "Mean", "Maximum", "Minimum", amp_str])

In [19]:
stats_df

,WWTP,Mean,Maximum,Minimum,Amplitude (max-min)(%)
0,1,0.714,0.912,0.552,36.000
1,2,0.784,0.966,0.630,33.600
2,3,0.876,1.000,0.650,35.000
3,4,0.875,1.000,0.684,31.600
4,5,0.779,0.966,0.622,34.400
5,6,0.536,1.000,0.433,56.700
6,7,0.556,0.686,0.446,24.000
7,Mean,0.731,0.933,0.574,35.900
8,SD,0.129,0.105,0.093,9.267


In [20]:
# Save statistics dataframe as csv file
stats_df.to_csv(path + "/statistics.csv",index=False)

In [21]:
# Convert Jupyter Notebook to PDF LaTeX file
!jupyter-nbconvert --to pdf "clip-max-ignore-zeros-asymmetric" --output-dir "../../results/10/asymmetric"

[NbConvertApp] Converting notebook clip-max-ignore-zeros-asymmetric.ipynb to pdf
[NbConvertApp] Writing 44332 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 47216 bytes to ../../results/10/asymmetric/clip-max-ignore-zeros-asymmetric.pdf
